In [ ]:
from AMC_python import AMC
from TimeTagger import *
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import datetime
import pandas as pd


# Initialize TCSPC
tt = createTimeTagger()

def g2(t):
"""
Function for measuring normalized coincidences on 2 channels of the TCSPC fed by SPAD outputs
Arg: t - acqusition time in picoseconds
Return: numpy.array of normalized coincidences with length = no. of bins
Displays real-time output of the measurement 
Based on the Correlation measurement class of Swabian API
"""
    tt.getOverflowsAndClear()
    channel_1 = 1 # SPAD channel 1
    channel_2 = 2 # SPAD channel 2
    n_bins = 1000 # no. of bins of the bi-directional histogram
    binwidth = 1000 #b inwidth of histogram bins in ps
    corr = Correlation(tt,channel_1,channel_2,binwidth,n_bins) # Measurement class of Swabian API
    corr.startFor(t) 
    while corr.isRunning():
        carr = corr.getDataNormalized() # Acquire coincidences for time 't'
        
        # For real-time display in Jupyter
        clear_output(wait=True)
        plt.plot(range(len(carr)),carr[range(len(carr))])
        plt.show()
    return carr 
        
def cntrate(t,x):
"""
Function for measuring countrate (counts/s or cps) for 'x' channel
Args: t - acqusition time in picoseconds; x - channel no.
Return: numpy.array of len 1 containing countrate from channel x
Displays real-time output of the measurement 
Based on the Countrate measurement class of Swabian API
"""
    cr = Countrate(tt, [x])  # Measurement class of Swabian API
    cr.startFor(t) # Set acquisition time
    
    while cr.isRunning():
        countrates = cr.getData() # Acquire countrate data
        print(countrates,end='\r')
    return countrates # Returns countrate from x

# IP Address of the AMC driver
IP = "192.168.1.1"

# Setup connection to AMC
amc = AMC.Device(IP)
amc.connect()

# Activate axes
axis0 = 0 # Axis 1
amc.control.setControlOutput(axis0, True)
axis1 = 1 # Axis 2
amc.control.setControlOutput(axis1, True)



def raster_scan(x_move,y_move,x_step,y_step,t,mode,thresh):
"""
Function for performing scanning g2 measurements
Args: x_move, y_move = total no. of steps to move in x, y directions, respectively
x_step, y_step = no. of steps to move in x or y direction with one iteration (x,y resolution)
t = acquisition time of g2 function
mode = Boolean; whether to conduct g2 with scan or not
thresh = intensity threshold in cps above which g2 will be measured

Return: list of tuples: (x,y,countrate, min. g2 value, g2 numpy.array);
at pixels where g2 is not measured, zeroes are returned
Displays real-time output of the measurement (plots intensity matrix and min g2 (g2(0) values) matrix)
"""
    
    pos_xy = [] # Initialize list of tuples
    coincs = np.zeros((y_move,x_move+1)) # Initialize coincidences map plot to zeroes
    cntr = np.zeros((y_move,x_move+1))   # Initialize counts map plot to zeroes
    amc.control.setControlAmplitude(axis1,25000) # y axis voltage is applied
    # m = 0
    plt.subplots(1,2,figsize=(15,15))
    for i in range(y_move): # For loop for y motion
        x = i
        amc.move.setNSteps(axis1, False, y_step) ## Moves Up == False
                                                 ## Moves Down == True

        g2y = np.zeros(1000) # Initialize g2 np.array to zeroes of len == no. of bins
        
        cnty = cntrate(0.1e12,1) # Acquire counts from y position for 0.1e12 seconds
        if mode == True and cnty > thresh:
            g2y = g2(t) # Measures g2 for t picoseconds
            
        # Store x, y position, countrate, min. g2, g2 np.array whwerever measured (if not measured min. g2 and g2 array are zeroes)
        pos_xy.append((amc.move.getPosition(axis0),amc.move.getPosition(axis1),cnty,np.amin(g2y),g2y)) 
        clear_output(wait=True)
        # Reshape matrices to plot in real time and mimic raster scan motion
        if x%2 == 1:
            cntr[i,x_move] = cnty 
            coincs[i,x_move] = np.amin(g2y)
        else:
            cntr[i,0] = cnty 
            coincs[i,0] = np.amin(g2y)

        # Real-time display
        plt.figure(figsize=(20,10))
        plt.subplot(1, 2, 1)
        img = plt.imshow(cntr)
        plt.colorbar(img)

        plt.subplot(1, 2, 2)
        img2 = plt.imshow(coincs)
        plt.colorbar(img2)
        plt.show()


        # For loop for x motion
        for j in range(x_move):
            # IF x motion in fwd and back direction is not the same for a particular amplitude of actuator signal
            if x%2 == 1:
                ### BACKWARD
                amc.control.setControlAmplitude(axis0,22600) ## Changes amplitude in back (x avis) 
            else:
                ### FORWARD
                amc.control.setControlAmplitude(axis0,27301)  ## Changes amplitude in fwd (x avis) 
                
            amc.move.setNSteps(axis0, x%2, x_step)  ## Moves in x
            g2x = np.zeros(1000)  # Initialize g2 np.array to zeroes of len == no. of bins
            
            cntx = cntrate(0.1e12,1) # Acquire counts from x position for 0.1e12 seconds
            if mode == True and cntx > thresh:
                g2x = g2(t)
             # Store x, y position, countrate, min. g2, g2 np.array whwerever measured (if not measured min. g2 and g2 array are zeroes)
            pos_xy.append((amc.move.getPosition(axis0),amc.move.getPosition(axis1),cntx,np.amin(g2x),g2x))
            
            clear_output(wait=True)
            
            # Reshape matrices to plot in real time and mimic raster scan motion
            if x%2 == 0:
                cntr[i,j+1] = cntx
                coincs[i,j+1] = np.amin(g2x)
            else:
                cntr[i,x_move-(j+1)] = cntx              
                coincs[i,x_move-(j+1)] = np.amin(g2x)
                
            # Real-time display
            plt.figure(figsize=(20,10))
            plt.subplot(1, 2, 1)
            
            img = plt.imshow(cntr)
            plt.colorbar(img)
            
            plt.subplot(1, 2, 2)
            
            img2 = plt.imshow(coincs)
            plt.colorbar(img2)
            plt.show()

    
    return pos_xy # Returns final list of tuples




x_step = 1 # x step
y_step = 1 # y step
thresh = 120000 # threshold intensity
modeg2 = False # mode of measurement incl. g2 or not
t_g2 = 30e12 # g2 acquisition time in ps
y_move = int(input("No. of steps in y:")) # user input for total x steps
x_move = int(input("No. of steps in x:")) # User input for total y steps
save = input("Do you want to save the xy data for this scan (y or n)?") 


p = raster_scan(x_move,y_move,x_step,y_step,t_g2,modeg2,thresh) # measurement data stored in p


# Saving protocol
date = datetime.today().strftime('%Y%m%d') 
folder_name = datetime.today().strftime('%B-%Y') 
if save == 'y':
    scan_num = int(input("Scan num:"))
    sample = input("Sample name (without space):")
    df = pd.DataFrame(p, columns = ['X_data','Y_data','Counts','Min. g2','Coincidence',])
    path = 'D:\\Data\\'+folder_name+'\\'+date
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

    df.to_csv(path+'\\'+sample+'_scan'+str(scan_num)+'_g2.csv', index=False)
    xy = [(i[0],i[1]) for i in p]
    print('Your file is saved as:',path+'\\'+sample+'_scan'+str(scan_num)+'_g2.csv')    
